In [1]:
import shutil

source_file_path = '.\..\..\.env'

destination_file_path = '.'

shutil.copy(source_file_path, destination_file_path)

'.\\.env'

In [2]:
import django
import os
import sys
project_path = '../../'  # Adjust this to your actual project path
sys.path.append(project_path)
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'oogway.settings')
django.setup()

import re
from asgiref.sync import sync_to_async
from telethon.tl.types import  Message, PeerChannel
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.sync import TelegramClient, events
from tqdm import tqdm
from datetime import datetime, timedelta, timezone
from PostAnalyzer.models import (
    Channel,
    EntryTarget,
    Market,
    Post,
    PostStatus,
    Predict,
    Symbol,
    TakeProfitTarget,
    SettingConfig,
    PositionSide,
    MarginMode,
)
import warnings
from django.forms.models import model_to_dict
from PostAnalyzer.Utility.utils import returnSearchValue,convertToJsonFile,subtractTime,load_json,print_colored

from dotenv import dotenv_values
from PostAnalyzer.Utility.findSameSignal import findSameSignal
from Shared.findRiskToReward import findRiskToReward
from Shared.findShortOrderStat import findShortOrderStat
from Shared.findLongOrderStat import findLongOrderStat
from Shared.findError import findError

config = dotenv_values(".env")

api_id = config["api_id"]
api_hash = config["api_hash"]

username = config["username"]

error_msg = []


allPredicts = []

historic_json_path = "./../historic-json"


# Disable all warnings
warnings.filterwarnings("ignore")

## is Predict Msg?

In [3]:
def isPredictMsg(msg):
    patterns = [
        r"Symbol:\s*#?([A-Z0-9]+)[/\s]?USDT",
        r"Take-Profit Targets:([\s\S]+?)(StopLoss|Description)",
        r"Entry (Targets|Price):([\s\S]+?)Take-Profit",
        r"Market:\s*([A-Z]+)",
        r"(StopLoss|Description):\s*([\d.]|\w+)",
    ]

    # Check if all patterns have a value
    return all(re.search(pattern, msg, re.IGNORECASE) for pattern in patterns)

## Symbol

In [4]:
async def findSymbol(msg):
    symbol = re.search(r"Symbol:\s*#?([A-Z0-9]+)[/\s]?USDT", msg, re.IGNORECASE)
    
    try:
        return await sync_to_async(Symbol.objects.get)(asset=returnSearchValue(symbol).upper())
    except:
        return None

## Market

In [5]:
async def findMarket(msg):
    market_match = re.search(r"Market:\s*([A-Z]+)", msg, re.IGNORECASE)
    
    try:
        market_value = await sync_to_async(Market.objects.get)(name=returnSearchValue(market_match).upper())
        return market_value
    except:
        return None


## Position

In [6]:
async def findPosition(msg):
    position_match = re.search(r"Position:\s*([A-Z]+)", msg)
    
    try:
        position_value = await sync_to_async(PositionSide.objects.get)(name=returnSearchValue(position_match).upper())
        return position_value
    except:
        return None

## Leverage and Margin Mode

In [7]:
async def findLeverage(msg):
    leverage_match = re.search(r"Leverage:\s*(Isolated|Cross)\s*(\d+x)", msg, re.IGNORECASE)
    if leverage_match:
        leverage_type = await sync_to_async(MarginMode.objects.get)(name=returnSearchValue(leverage_match).upper())   
        leverage_value = int(leverage_match.group(2).lower().replace("x",""))    
    else:
        leverage_type = None
        leverage_value = None
    
    return leverage_type, leverage_value

## StopLoss

In [8]:
def findStopLoss(msg):
   msg = msg.replace(",","")

   return float(returnSearchValue(re.search(r"StopLoss:\s*([\d.]+)", msg)))

## Entry Targets

In [9]:
def findEntryTargets(msg):
    msg = msg.replace(",","")
    match = re.search(r"Entry Targets:([\s\S]+?)Take-Profit", msg, re.IGNORECASE)
    match1 = re.search(r"Entry Price:([\s\S]+?)Take-Profit", msg, re.IGNORECASE)
    final = match if match else match1
    if final:
        extracted_data = returnSearchValue(final)
        return [float(x.strip()) for i, x in enumerate(re.findall(r"(\d+\.\d+|\d+)", extracted_data))  if i % 2 == 1]

## Take Profits

In [10]:
def findTakeProfits(msg):
    msg = msg.replace(",","")
    match = re.search(r"Take-Profit Targets:([\s\S]+?)(StopLoss|Description)", msg, re.IGNORECASE)
    if match:
        extracted_data = returnSearchValue(match)
        return [float(x.strip()) for i, x in enumerate(re.findall(r"(\d+\.\d+|\d+)", extracted_data)) if i % 2 == 1]
        

## Check if post is a Cancel predict or not


In [11]:
async def isCancel(post):
    if post is None or post.reply_to_msg_id is None:
        return False
    
    try:
        # cancel_before_patterns = [
        #     r"Cancelled",
        #     r"achieved",
        #     r"entering",
        # ]
        manually_cancel_patterns = [
            r"Cancelled",
            r"Manually",
        ]  
        cancel_pattern =[
            r"کنسل"
        ]


        # check = all(re.search(pattern, post.message, re.IGNORECASE) for pattern in cancel_before_patterns)
        check1 = all(re.search(pattern, post.message, re.IGNORECASE) for pattern in manually_cancel_patterns)
        check2 = all(re.search(pattern, post.message, re.IGNORECASE) for pattern in cancel_pattern)
        
        if check1 or check2 :
            
            predict = await sync_to_async(Predict.objects.get)(
                post__message_id=post.reply_to_msg_id
            )
            
            status_value = await sync_to_async(PostStatus.objects.get)(name="CANCELED")
        
            predict.status = status_value
            predict.profit = 0
        
            await sync_to_async(predict.save)()
            return True
            
        else:
            return False
        
    except:
        # error_msg.append(model_to_dict(post))
        return False


## Find important parts of a predict message such as symbol or entry point

In [12]:
async def predictParts(string, post):
    if string is None or post is None:
        return None
    
    try:
        should_check = True
        settings = await sync_to_async(SettingConfig.objects.get)(id=1)
        

        # symbol
        symbol_value:Symbol = await findSymbol(string)

        # market
        market_value= await findMarket(string)
        isSpot = market_value.name == "SPOT"

        # position, leverage, marginMode
        position_value = None
        leverage_value = None
        marginMode_value = None
        if not isSpot:
            position_value = await findPosition(string)
            marginMode_value, leverage_value = await findLeverage(string)
        else:
            position_value= await sync_to_async(PositionSide.objects.get)(name="BUY")
        
        # stopLoss
        stopLoss_value = findStopLoss(string)

        # entry targets
        entry_targets_value = findEntryTargets(string)

        # status    
        status_value = await sync_to_async(PostStatus.objects.get)(name="PENDING")

        # take_profit targets
        take_profit_targets_value = findTakeProfits(string)

        is_error = findError(position_value.name, take_profit_targets_value, entry_targets_value, stopLoss_value)
        if is_error:
            status_value = await sync_to_async(PostStatus.objects.get)(name="ERROR")
            should_check = False
            
        # return Error RR < 0.2
        # TODO will add to setting, to get min RR
        RR = findRiskToReward(entry_targets_value, take_profit_targets_value, stopLoss_value)
        
        if RR < 0.2 or RR > 4:
            status_value = await sync_to_async(PostStatus.objects.get)(name="ERROR")
            should_check = False
        
        # return false if signal already exists
        is_same_signal = await findSameSignal(post.date, symbol_value,market_value,position_value, leverage_value, marginMode_value,stopLoss_value,
                                    take_profit_targets_value,entry_targets_value,post.channel.id)
        if is_same_signal:
            return False
        
        print(symbol_value.name, position_value.name)
        isSHORT = position_value.name == "SHORT"
        first_entry_value = entry_targets_value[0]
        
        if should_check:
            if not isSHORT:
                stat = await findLongOrderStat(stop_loss=stopLoss_value,
                                entry_price=entry_targets_value,
                                symbol=symbol_value,
                                take_profit=take_profit_targets_value,
                                start_timestamp= int(post.date.timestamp() * 1000))
            elif isSHORT:
                stat = await findShortOrderStat(stop_loss=stopLoss_value,
                                entry_price=entry_targets_value,
                                symbol=symbol_value,
                                take_profit=take_profit_targets_value,
                                start_timestamp= int(post.date.timestamp() * 1000))
        else:
            stat = {"tps": [], "entry_reached": [], "stop_loss_reached": None}
            
        print(stat)
        stat['name'] = f'{symbol_value.name}, {position_value.name}'
        allPredicts.append(stat)
        print_colored("********************************************************************************************************************","#0f0")
        tps_length = len(stat['tps'])
        profit_value = 0

        if stat['stop_loss_reached']:
            if tps_length > 0:
                status_value = await sync_to_async(PostStatus.objects.get)(name="FAILED WITH PROFIT", type=tps_length)
            else:
                status_value = await sync_to_async(PostStatus.objects.get)(name="FAILED")
            profit_value = round(((float(stopLoss_value)/float(first_entry_value))-1)*100*float(leverage_value) * (-1 if isSHORT else 1), 5)
        else:
            if tps_length == len(take_profit_targets_value):
                status_value = await sync_to_async(PostStatus.objects.get)(name="FULLTARGET")
                profit_value = round(abs(((take_profit_targets_value[tps_length-1]/first_entry_value)-1)*100*leverage_value), 5)

            elif tps_length > 0:
                status_value = await sync_to_async(PostStatus.objects.get)(name="SUCCESS", type=tps_length)
                profit_value = round(abs(((take_profit_targets_value[tps_length-1]/first_entry_value)-1)*100*leverage_value), 5)
            
        

        # set predict object to DB
        PredictData = {
            "post": post,
            "date": post.date,
            "symbol": symbol_value,
            "position": position_value,
            "market": market_value,
            "leverage": leverage_value,
            "stopLoss": stopLoss_value,
            "margin_mode": marginMode_value,
            "profit": profit_value,
            "status": status_value,  # PENDING = 1
            "order_id": None,
        }
        newPredict = Predict(**PredictData)

        # set entry value objects to DB
        if entry_targets_value:
            entriesLen = len(stat['entry_reached'])
            for i, value in enumerate(entry_targets_value):
                isActive = i < entriesLen 
                date = None
                period = None
                if isActive:
                    time = int(stat['entry_reached'][i]['time'])
                    date = datetime.fromtimestamp(time/ 1000)
                    period = subtractTime(time, int(post.date.timestamp() * 1000))
                entryData = EntryTarget(
                    **{
                        "post": post,
                        "index": i,
                        "value": value,
                        "active": isActive,
                        "period": period,
                        "date": date,
                    }
                )
                await sync_to_async(entryData.save)()
        
        # # set tp value objects to DB
        if take_profit_targets_value:
            tpLen = len(stat['tps'])

            for i, value in enumerate(take_profit_targets_value):
                isActive = i < tpLen 
                date = None
                period = None
                if isActive:
                    time = int(stat['tps'][i]['time'])
                    date = datetime.fromtimestamp(time/ 1000)
                    period = subtractTime(time, int(post.date.timestamp() * 1000))

                takeProfitData = TakeProfitTarget(
                    **{
                        "post": post,
                        "index": i,
                        "value": value,
                        "active": isActive,
                        "period": period,
                        "profit": round(abs(((value/first_entry_value)-1)*100*leverage_value), 5),
                        "date": date,
                    }
                )
                await sync_to_async(takeProfitData.save)()


        await sync_to_async(newPredict.save)()
        return newPredict
    except:
        error_msg.append(model_to_dict(post))
        
        return False
     

## Find message type and save to DB

In [13]:
async def extract_data_from_message(message):
    if isinstance(message, Message):
        is_predict_msg = isPredictMsg(message.message)
        channel = await sync_to_async(Channel.objects.get)(
            channel_id=message.peer_id.channel_id
        )
        PostData = {
            "channel": channel if channel else None,
            "date": message.date,
            "message_id": message.id,
            "message": message.message,
            "reply_to_msg_id": message.reply_to.reply_to_msg_id
            if message.reply_to
            else None,
            "edit_date": message.edit_date,
            "is_predict_msg": is_predict_msg,
        }
        post = Post(**PostData)

        await sync_to_async(post.save)()
        # predict msg
        if is_predict_msg:
            await predictParts(message.message, post)
        # # cancel msg
        elif await isCancel(post):
            pass
        return PostData
    else:
        return None



## Get channel history

In [14]:
client = await TelegramClient(username, api_id, api_hash).start()
peer_channel =  PeerChannel(int(config["CHANNEL_FEYZ"]))
feyzian_channel = await client.get_entity(peer_channel)

offset_id = 0
limit = 100
all_messages = []
total_messages = 0
total_count_limit = 0
# اول فروردین = 2024/3/20
end_date = datetime(2024, 3, 20, tzinfo=timezone.utc)
# اول خرداد = 2024/5/21
# end_date = datetime(2024, 5, 21, tzinfo=timezone.utc)
# start_date = datetime(2024, 5, 29, tzinfo=timezone.utc)
# اول تیر = 2024/6/21
# start_date = datetime(2024, 6, 21, tzinfo=timezone.utc)
# end_date = datetime(2024, 7, 15, tzinfo=timezone.utc)

shouldStop = False
while not shouldStop:
    print("Current Offset ID is:", offset_id, "; Total Messages:", total_messages)
    history = await  client(
    GetHistoryRequest(
            peer=feyzian_channel,
            offset_id=offset_id,
            offset_date=None,
            # offset_date=start_date,
            add_offset=0,
            limit=limit,
            max_id=0,
            min_id=0,
            hash=0,
        )
    )

    if not history.messages:
        break

    messages = history.messages
    for message in tqdm(messages):
        message_date = message.date.replace(tzinfo=timezone.utc)
        # if start_date < message_date:
        #     continue

        if message_date < end_date:
            shouldStop = True
            break
        all_messages.append(message)

    offset_id = messages[len(messages) - 1].id
    total_messages = len(all_messages)
    if total_count_limit != 0 and total_messages >= total_count_limit:
        break

await client.disconnect()

for msg in reversed(all_messages):
    await extract_data_from_message(msg)



# print(error_msg)
convertToJsonFile(error_msg, "error_msg", ".") 
# convertToJsonFile(allPredicts, "allPredicts", ".")  
{"open": "38.586", "close": "38.649", "high": "38.649", "low": "38.585", "volume": "2865.00", "time": 1716635700000}


Current Offset ID is: 0 ; Total Messages: 0


100%|██████████| 100/100 [00:00<?, ?it/s]


Current Offset ID is: 12376 ; Total Messages: 100


100%|██████████| 100/100 [00:00<?, ?it/s]


Current Offset ID is: 12266 ; Total Messages: 200


...

ETH-USDT SHORT
2736.8 [2692.68, 2721.81] ETH-USDT [2680.08, 2666.34, 2651.06, 2634.51, 2609.11] 1723493498000


{'tps': [{'open': '2684.52', 'close': '2678.30', 'high': '2684.59', 'low': '2674.89', 'volume': '824.65', 'time': 1723495860000}, {'open': '2667.69', 'close': '2670.80', 'high': '2671.05', 'low': '2665.87', 'volume': '441.39', 'time': 1723496760000}], 'entry_reached': [{'open': '2692.56', 'close': '2700.38', 'high': '2702.00', 'low': '2690.25', 'volume': '814.41', 'time': 1723495320000}, {'open': '2717.00', 'close': '2724.20', 'high': '2727.78', 'low': '2715.11', 'volume': '701.51', 'time': 1723504680000}], 'stop_loss_reached': {'open': '2724.51', 'close': '2745.86', 'high': '2748.72', 'low': '2724.51', 'volume': '3033.51', 'time': 1723504740000}}


ADA-USDT SHORT
0.346 [0.3348, 0.3434] ADA-USDT [0.332, 0.3279, 0.3237, 0.3192, 0.3111] 1723539537000


{'tps': [{'open': '0.33207', 'close': '0.33213', 'high': '0.33218', 'low': '0.33200', 'volume': '53271.00', 'time': 1723542300000}], 'entry_reached': [{'open': '0.33494', 'close': '0.33485', 'high': '0.33499', 'low': '0.33484', 'volume': '49987.00', 'time': 1723539540000}], 'stop_loss_reached': None}


{'open': '38.586',
 'close': '38.649',
 'high': '38.649',
 'low': '38.585',
 'volume': '2865.00',
 'time': 1716635700000}